In [3]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [30]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [5]:
from secrets import IEX_CLOUD_API_TOKEN

In [13]:
def get_stock_info(ticker):
    api_url = f"https://sandbox.iexapis.com/stable/stock/{ticker}/quote?token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(api_url)
    if data.status_code != 200:
        return None
    return data.json()
    
# symbol = 'AAPL'
# api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}"

In [25]:
cols = ['Ticker', 'Stock Price', 'Market Cap', '#Shares to buy']
dictionary = {col: list() for col in cols}

In [26]:
for sym in stocks['Ticker']:
    data = get_stock_info(sym)
    if data:
        dictionary['Ticker'].append(sym)
        dictionary['Stock Price'].append(data['latestPrice'])
        dictionary['Market Cap'].append(data['marketCap'])
        dictionary['#Shares to buy'].append('N/A')

In [28]:
df = pd.DataFrame(dictionary)

In [29]:
df

,Ticker,Stock Price,Market Cap,#Shares to buy
0,A,116.69,3.481776e+10,N/A
1,AAL,13.19,8.420363e+09,N/A
2,AAP,175.22,1.035976e+10,N/A
3,AAPL,132.28,2.217297e+12,N/A
4,ABBV,139.00,2.480638e+11,N/A
...,...,...,...,...
496,YUM,109.70,3.235926e+10,N/A
497,ZBH,103.10,2.252222e+10,N/A
498,ZBRA,293.17,1.541337e+10,N/A
499,ZION,53.15,8.252822e+09,N/A


## Use Batch API calls

In [64]:
cols = ['Ticker', 'Stock Price', 'Market Cap', '#Shares to buy']
dictionary = {col: list() for col in cols}

In [65]:
batches = [stocks[:86]['Ticker'], stocks[86: 192]['Ticker'], stocks[192:277]['Ticker'], stocks[277: 359]['Ticker'], stocks[359: 444]['Ticker'], stocks[444:]['Ticker']]

In [66]:
def batch_calls(tickers):
    tickers = ','.join(list(tickers))
    batch_url = f"https://sandbox.iexapis.com/stable/stock//market/batch?symbols={tickers}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_url)
    if data.status_code != 200:
        return None
    return data.json()

In [67]:

for batch in batches:
    data = batch_calls(batch)
    
    for ticker in batch:
        if ticker in data:
            dictionary['Ticker'].append(ticker)
            dictionary['Stock Price'].append(data[ticker]['quote']['latestPrice'])
            dictionary['Market Cap'].append(data[ticker]['quote']['marketCap'])
            dictionary['#Shares to buy'].append('N/A')
        

In [68]:
df = pd.DataFrame(dictionary)

In [77]:
portfolio_size = 10e6 # can change, just an example now
# equal size at first, split it between all companies evenly
capital_per_company = portfolio_size/df.shape[0]

df['#Shares to buy'] = capital_per_company//df['Stock Price']